In [2]:
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri, FloatVector, ListVector
import pandas as pd
from subprocess import call

In [7]:
ASE_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/ASE/nano_merge_ASE_SNP.csv"
m6A_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/m6A/nano_merge_m6A_SNP.csv"
pseU_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/pseU/nano_merge_pseU_SNP.csv"
stability_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/stability/nano_merge_stability_SNP.csv"
isoform_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/isoform/nano_merge_isoform_SNP.csv"
polyA_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/apa/nano_merge_polyA_SNP.csv"
promoter_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/promoter/nano_merge_promoter_SNP.csv"
tss_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/tss/nano_merge_tss_SNP.csv"

gene_bed_file = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/stability/gene_merge.bed"
outdir = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/colco"


In [8]:
ASE_df = pd.read_csv(ASE_file,usecols = ['rsID','chrom','snp_pos_1base','strand','p_value','beta','SE'])

In [9]:
ASE_df

,chrom,strand,snp_pos_1base,rsID,p_value,beta,SE
0,chr1,+,949608,rs1921,1.031165e-138,-1.264128,0.016382
1,chr1,+,949831,rs116002608,1.565662e-05,-4.493839,0.034232
2,chr1,+,949889,rs3829742,2.581353e-12,-4.404020,0.033459
3,chr1,+,990280,rs4275402,7.955933e-01,-1.828340,0.184928
4,chr1,+,990380,rs3121561,3.931862e-01,-2.041027,0.182121
...,...,...,...,...,...,...,...
7322,chr9,+,140167985,rs28740782,1.123909e-01,-0.277248,0.086732
7323,chr9,+,140175422,rs117569577,8.930123e-01,-3.624491,0.387309
7324,chr9,+,140176814,rs112239920,6.573195e-03,-1.223089,0.100219
7325,chr9,+,140446707,rs149264137,1.175815e-09,-5.409122,0.183889


In [ ]:
import numpy as np
import pandas as pd
from rpy2.robjects import pandas2ri, r
from rpy2.robjects.packages import importr

# 激活pandas与R的转换
pandas2ri.activate()

# 导入Primo包
primo = importr('Primo')

# 准备数据
n_snps = 10000
n_studies = 4

# 创建模拟数据（请用您的实际数据替换这部分）
betas = np.random.normal(0, 0.1, (n_snps, n_studies))
sds = np.random.uniform(0.05, 0.2, (n_snps, n_studies))
dfs = [105, 105, 104, 104]  # 根据您的情况设置
pvalues = np.random.uniform(0, 1, (n_snps, n_studies))

# 将NumPy数组转换为R矩阵
r_betas = pandas2ri.py2rpy(pd.DataFrame(betas))
r_sds = pandas2ri.py2rpy(pd.DataFrame(sds))
r_dfs = r.c(*dfs)
r_pvalues = pandas2ri.py2rpy(pd.DataFrame(pvalues))

# 设置alt_props
r_alt_props = r.c(1e-2, 2e-2, 3e-2, 5e-2)

# 运行Primo
primo_results = primo.Primo(betas=r_betas, sds=r_sds, dfs=r_dfs, alt_props=r_alt_props)

# 提取结果
post_prob = pandas2ri.rpy2py(primo_results.rx2('post_prob'))
pis = pandas2ri.rpy2py(primo_results.rx2('pis'))

# 分析共定位
postprob_atLeastN = primo.collapse_pp_num(post_prob=post_prob)
common_qtl_snps = np.where(postprob_atLeastN[:, 1] > 0.5)[0]  # PP_ge2 > 0.5

# 计算FDR
def calc_fdr(pp, thresh):
    r_pp = pandas2ri.py2rpy(pp)
    return primo.calc_fdr(r_pp, thresh=thresh)[0]

fdr_grid = [calc_fdr(postprob_atLeastN[:, 1], th) for th in np.arange(0.9, 0.3, -0.1)]

# 打印结果
print("Number of SNPs with potential co-localization:", len(common_qtl_snps))
print("FDR at different thresholds:", fdr_grid)

# 可视化结果（使用Python的matplotlib）
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.imshow(post_prob[common_qtl_snps, :], aspect='auto', cmap='YlOrRd')
plt.colorbar(label='Posterior Probability')
plt.xlabel('QTL Studies')
plt.ylabel('SNPs')
plt.title('Co-localization Heatmap')
plt.tight_layout()
plt.show()

In [ ]:
input <- merge(eqtl, gwas, by="rs_id", all=FALSE, suffixes=c("_eqtl","_gwas"))
result <- coloc.abf(dataset1=list(pvalues=input$pval_nominal_gwas, type="cc", s=0.33, N=50000), dataset2=list(pvalues=input$pval_nominal_eqtl, type="quant", N=10000), MAF=input$maf)

need_result=result$results %>% filter(SNP.PP.H4 > 0.95)

In [ ]:
# 启用pandas和R的数据框之间的转换
pandas2ri.activate()

# 加载R包
coloc = importr('coloc')

# 创建pandas数据框
data1 = pd.DataFrame({
    'snp': ['rs1', 'rs2', 'rs3'],
    'beta': [0.2, 0.4, 0.6],
    'varbeta': [0.02, 0.04, 0.06],
    'MAF': [0.1, 0.2, 0.3],
    'N': [1000, 1000, 1000],
    'type': ['quant', 'quant', 'quant']
})

data2 = pd.DataFrame({
    'snp': ['rs1', 'rs2', 'rs3'],
    'beta': [0.3, 0.5, 0.7],
    'varbeta': [0.03, 0.05, 0.07],
    'MAF': [0.1, 0.2, 0.3],
    'N': [1000, 1000, 1000],
    'type': ['quant', 'quant', 'quant']
})

# 将pandas数据框转换为R数据框
rdf1 = pandas2ri.py2rpy(data1)
rdf2 = pandas2ri.py2rpy(data2)

# 运行coloc.abf函数进行共定位分析
result = coloc.coloc_abf(dataset1=rdf1, dataset2=rdf2)

# 将结果转换回pandas数据框
result_df = pandas2ri.rpy2py(result)
print(result_df)


In [ ]:
# 假设你有一个输入字典input包含pval_nominal_gwas, pval_nominal_eqtl, 和maf
input_data = {
    'pval_nominal_gwas': [0.01, 0.02, 0.05],  # 示例数据
    'pval_nominal_eqtl': [0.03, 0.04, 0.01],  # 示例数据
    'maf': [0.1, 0.2, 0.3]                    # 示例数据
}

# 将输入数据转换为R的向量
pval_nominal_gwas = FloatVector(input_data['pval_nominal_gwas'])
pval_nominal_eqtl = FloatVector(input_data['pval_nominal_eqtl'])
maf = FloatVector(input_data['maf'])

# 创建dataset1和dataset2
dataset1 = ListVector({
    'pvalues': pval_nominal_gwas,
    'type': 'cc',
    's': 0.33,
    'N': 50000
})

dataset2 = ListVector({
    'pvalues': pval_nominal_eqtl,
    'type': 'quant',
    'N': 10000
})

# 运行coloc.abf函数
result = coloc.coloc_abf(dataset1=dataset1, dataset2=dataset2, MAF=maf)

# 将结果转换为Python数据结构（如需要）
result_df = pandas2ri.rpy2py(result)
print(result_df)
